In [1]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv("task1_allData.csv")
test_public = pd.read_csv("task1_public_test_data_.csv")
test = pd.read_csv("task1_private_test_data.csv")

In [3]:
test = pd.merge(test_public, test, how = "outer")

In [4]:
y_train = train["IsCorrect"]
x_train = train.drop(["IsCorrect", "AnswerId"], axis=1)
y_test = test["IsCorrect"]
x_test = test.drop(["IsCorrect", "AnswerId"], axis=1)

In [5]:
x_train["SchemeOfWorkId"] = x_train["SchemeOfWorkId"].astype(int)
x_train["PremiumPupil"] = x_train["PremiumPupil"].astype(int)
x_train["Confidence"] = x_train["Confidence"].astype(int)

x_test["SchemeOfWorkId"] = x_test["SchemeOfWorkId"].astype(int)
x_test["PremiumPupil"] = x_test["PremiumPupil"].astype(int)
x_test["Confidence"] = x_test["Confidence"].astype(int)

In [6]:
model = CatBoostClassifier(
    iterations=4,
    cat_features=["UserId", "QuestionId", "QuizId", "GroupId", "SchemeOfWorkId",
                  "Gender", "PremiumPupil", "Confidence", "MathType", "SchoolShift"],
    verbose=2
)

In [7]:
model.fit(x_train, y_train)

y_predict = model.predict(x_test)
print('private ', accuracy_score(y_test, y_predict))

Learning rate set to 0.5
0:	learn: 0.5671803	total: 4.5s	remaining: 13.5s
2:	learn: 0.5269787	total: 13.4s	remaining: 4.46s
3:	learn: 0.5224557	total: 16.6s	remaining: 0us
private  0.7302805697960884


In [8]:
print(classification_report(y_test, y_predict, target_names=["Wrong Answer", "Correct Answer"]))

                precision    recall  f1-score   support

  Wrong Answer       0.64      0.56      0.60   1415961
Correct Answer       0.77      0.83      0.80   2551002

      accuracy                           0.73   3966963
     macro avg       0.71      0.69      0.70   3966963
  weighted avg       0.72      0.73      0.73   3966963



In [9]:
df = pd.DataFrame(y_predict, columns =['CatBoost'])
df.to_csv("CatBoost1.csv", header=True, index=False)

In [6]:
import pickle
pickle.dump(model, open("CatBoost.sav", 'wb'))